# Get Data from Twitter


In [1]:
import twitter
import tweepy
from tweepy import OAuthHandler
consumer_key = "hEDGEdp4p2Tp8U6h5vpOGOpm9"
consumer_secret = "4C8P1YeOtsjebf6dYyTnRCWae3GWWoF9n6VLnQOKTEorlK3KbT"
access_token = "846992744690208768-QQKUada7IjhguoQyjrARQ99nCkHVMop"
access_token_secret = "5zfmqKV2eLPO2SU0XoW3DKtOUqwmvpPB1r7KoQ48OQnq8"
authorization = twitter.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)
t = twitter.Twitter(auth=authorization)

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth, proxy="127.0.0.1:8123")

for status in tweepy.Cursor(api.home_timeline).items(2):
    print (status.text)

50% of ur friends in their 20s vs. the other 50% of ur friends in their 20s https://t.co/Fc57ZUOkQt
open for a double surprise https://t.co/QQTjOWsVxD


In [2]:
import os
import json

output_filename = os.path.join(".", "data", "python_tweets.json")
with open(output_filename, 'a') as output_file:
    search_results = api.search(q="python", count=100)
    for tweet in search_results:
        if 'text' in tweet._json:
            print(tweet._json['text'])
            output_file.write(json.dumps(tweet._json))
            output_file.write("\n\n")

RT @machinelearnbot: Natural Language Processing with Deep Learning in Python https://t.co/X0yOxiezE8 #machinelearning
@NationalTheatre for the faintly surreal St George where Monty Python meets Medieval Mystery play in riff on all things English.  Diverting.
Disruptors: Entrepreneurs &amp; The Escape From Corporate America here  https://t.co/8SEKLMQtvg #python #programming @pythonbot_
Python: Beginner's Guide To Programming Code With Python  https://t.co/3w2x9otZqb #software #development #python @pythonbot_
RT @Amaka_Ekwo: Alert! 
Nigerian Army starts operation python poison across BiafraLand; forcefully injecting Biafran children with virus. #…
俺の20mのフルアダルトの胴回りの太さはハンパねぇな。そう、大体直径1mくらいになっちゃうよ。
django-carrot 0.1.5a0: A RabbitMQ asynchronous task queue for Django. https://t.co/mOPgCLJCqw
puer 0.4.1:  https://t.co/oq6mpKqa9M
@YouTube C++でコアを作ってFEにラップして+Pythonでスタンドアロン化してるのね。FE挟んでるからＤＣＣツールを選ばずどこでも使えると。
ツール作る側からすれば最高の環境だな。
RT @Demoore90210: Confirmation!
Buhari intends to build the North; an

# Label Twitter Data

In [3]:
# Labelling the class values for the twitter dataset.
import os
input_filename = os.path.join(".", "data", "python_tweets.json")
classes_filename = os.path.join(".", "data", "python_classes.json")

In [4]:
import json
tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0:
            continue
        tweets.append(json.loads(line))
print("Loaded {} tweets".format(len(tweets)))

Loaded 400 tweets


In [5]:
tweet_sample = tweets
labels = []
if os.path.exists(classes_filename):
    with open(classes_filename) as inf:
        labels = json.load(inf)
print(len(labels))

101


In [6]:
def get_next_tweet():
    return tweet_sample[len(labels)]['text']


In [7]:
%%javascript
function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}

function load_next_tweet(){
   var code_input = "get_tweet()";
   var kernel = IPython.notebook.kernel;
   var callbacks = { 'iopub' : {'output' : handle_output}};
   kernel.execute(code_input, callbacks, {silent:false});
}

function handle_output(out){
   console.log(out);
   var res = out.content.data["text/plain"];
   $("div#tweet_text").html(res);
}


<IPython.core.display.Javascript object>

In [8]:
%%html
<div name="tweetbox">
    Instructions: Click in text box. Enter a 1 if the tweet is relevant, enter 0 otherwise.<br>
    Tweet: <div id="tweet_text" value="text"></div><br>
    <input type=text id="capture"></input><br>
</div>
<script>
function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}

function load_next_tweet(){
   var code_input = "get_next_tweet()";
   var kernel = IPython.notebook.kernel;
   var callbacks = { 'iopub' : {'output' : handle_output}};
   kernel.execute(code_input, callbacks, {silent:false});
}

function handle_output(out){
   console.log(out);
   var res = out.content.data["text/plain"];
   $("div#tweet_text").html(res);
}
$("input#capture").keypress(function(e) {
    console.log(e);
  if(e.which == 48) {
    // 0 pressed
    set_label(0);
    $("input#capture").val("");
  }else if (e.which == 49){
    // 1 pressed
    set_label(1);  
    $("input#capture").val("");
  }
});

load_next_tweet();
</script>

In [9]:
print(len(labels))

101


In [10]:
with open(classes_filename, 'w') as outf:
    json.dump(labels, outf)

# Classify Twitter Data

In [11]:
import json
tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0:
            continue
        tweets.append(json.loads(line)['text'])
print("Loaded {} tweets".format(len(tweets)))

Loaded 400 tweets


In [12]:
with open(classes_filename) as inf:
    labels = json.load(inf)

In [13]:
n_samples = min(len(tweets), len(labels))
sample_tweets = [t.lower() for t in tweets[:n_samples]]
labels = labels[:n_samples]

In [14]:
import numpy as np
y_true = np.array(labels)

In [15]:
print("{:.1f}% have class 1".format(np.mean(y_true == 1) * 100))

3.0% have class 1


In [16]:
from sklearn.base import TransformerMixin
from nltk import word_tokenize
# import nltk
class NLTKBOW(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [{word: True for word in word_tokenize(document)}
                 for document in X]

In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

/home/dlinking-lxy/more-space/pyworks/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
# nltk.download('punkt')
pipeline = Pipeline([('bag-of-words', NLTKBOW()),
                     ('vectorizer', DictVectorizer()),
                     ('naive-bayes', BernoulliNB())
                     ])
scores = cross_val_score(pipeline, sample_tweets, y_true, cv=10, scoring='f1')
print("Score: {:.3f}".format(np.mean(scores)))

/home/dlinking-lxy/more-space/pyworks/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/home/dlinking-lxy/more-space/pyworks/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/dlinking-lxy/more-space/pyworks/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/dlinking-lxy/more-space/pyworks/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no pr

Score: 0.000


In [19]:
scores

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])